# Import Libraries and Data

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time
from sklearn.metrics import f1_score, confusion_matrix, precision_score

# Problem 2a

In [2]:
# Load CIFAR-10 dataset to calculate mean and std
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())

# Calculate mean and std
imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformation with calculated mean and std
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Load CIFAR-10 dataset with normalization
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()

        # First convolutional layer
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

        # Second convolutional layer
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Shortcut connection
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

# Define ResNet-10 model
class ResNet10(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10, n_chans1=64):
        super(ResNet10, self).__init__()
        self.in_channels = n_chans1

        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(n_chans1)
        self.relu = nn.ReLU()
        self.layer1 = self._make_layer(block, n_chans1, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, n_chans1 * 2, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, n_chans1 * 4, num_blocks[2], stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(n_chans1 * 4, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# Instantiate the ResNet-10 model with ResNet blocks
model = ResNet10(ResidualBlock, [4, 3, 3]).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Training loop
num_epochs = 300
total_start_time = time.time()
for epoch in range(num_epochs):
    start_time = time.time()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    end_time = time.time()
    training_time = end_time - start_time

    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Training Time: {training_time:.2f} seconds')

total_end_time = time.time()
total_training_time = total_end_time - total_start_time
print(f'Total Training Time: {total_training_time:.2f} seconds')

# Testing the model
model.eval()
correct = 0
total = 0
all_predicted = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = correct / total
print(f'Final Test Accuracy: {accuracy * 100:.2f}%')

# Calculate and print F1 score, precision, and recall at the end
f1 = f1_score(all_labels, all_predicted, average='weighted')
precision = precision_score(all_labels, all_predicted, average='weighted')
recall = recall_score(all_labels, all_predicted, average='weighted')

print(f'Final F1 Score: {f1:.4f}')
print(f'Final Precision: {precision:.4f}')
print(f'Final Recall: {recall:.4f}')

# Calculate and print confusion matrix at the end
cm = confusion_matrix(all_labels, all_predicted)
print('Final Confusion Matrix:')
print(cm)

100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified
Epoch 1/300, Loss: 1.833100466014784, Training Time: 54.60 seconds
Epoch 11/300, Loss: 0.6835234948360097, Training Time: 44.71 seconds
Epoch 21/300, Loss: 0.18683619255109515, Training Time: 44.57 seconds
Epoch 31/300, Loss: 0.030607760501572923, Training Time: 44.43 seconds
Epoch 41/300, Loss: 0.013625685093731469, Training Time: 44.35 seconds
Epoch 51/300, Loss: 0.008510028143547942, Training Time: 44.29 seconds
Epoch 61/300, Loss: 0.004117418974822582, Training Time: 44.45 seconds
Epoch 71/300, Loss: 0.0031953321566181186, Training Time: 44.62 seconds
Epoch 81/300, Loss: 0.007624684968237381, Training Time: 44.45 seconds
Epoch 91/300, Loss: 0.011092630650057955, Training Time: 44.68 seconds
Epoch 101/300, Loss: 0.0029602833423817703, Training Time: 44.20 seconds
Epoch 111/300, Loss: 0.003523125251739636, Training Time: 44.40 seconds
Epoch 121/300, Loss: 0.0

NameError: name 'precision_score' is not defined

In [5]:
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score
f1 = f1_score(all_labels, all_predicted, average='weighted')
precision = precision_score(all_labels, all_predicted, average='weighted')
recall = recall_score(all_labels, all_predicted, average='weighted')

print(f'Final F1 Score: {f1:.4f}')
print(f'Final Precision: {precision:.4f}')
print(f'Final Recall: {recall:.4f}')

# Calculate and print confusion matrix at the end
cm = confusion_matrix(all_labels, all_predicted)
print('Final Confusion Matrix:')
print(cm)

Final F1 Score: 0.7401
Final Precision: 0.7404
Final Recall: 0.7406
Final Confusion Matrix:
[[784  26  47  16  15  12   8  10  48  34]
 [ 19 881   7   4   4   3   4   4   9  65]
 [ 69   4 617  56  86  56  66  23  14   9]
 [ 24   9  62 563  56 163  68  23   8  24]
 [ 25   3  77  61 672  34  57  64   7   0]
 [ 15   5  43 168  43 658   9  43   6  10]
 [ 11   5  45  58  43  30 788  10   7   3]
 [ 16   3  26  36  70  73   8 749   4  15]
 [ 79  31   6  13  10   1   9   5 827  19]
 [ 23  59   4   4   7   6   5   5  20 867]]


# Problem 2b

Weight Decay with Lambda of 0.001

In [6]:
# Load CIFAR-10 dataset to calculate mean and std
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())

# Calculate mean and std
imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformation with calculated mean and std
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Load CIFAR-10 dataset with normalization
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()

        # First convolutional layer
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

        # Second convolutional layer
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Shortcut connection
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

# Define ResNet-10 model
class ResNet10(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10, n_chans1=64):
        super(ResNet10, self).__init__()
        self.in_channels = n_chans1

        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(n_chans1)
        self.relu = nn.ReLU()
        self.layer1 = self._make_layer(block, n_chans1, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, n_chans1 * 2, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, n_chans1 * 4, num_blocks[2], stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(n_chans1 * 4, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# Instantiate the ResNet-10 model with ResNet blocks
model = ResNet10(ResidualBlock, [4, 3, 3]).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, weight_decay=0.001)

# Training loop
num_epochs = 300
total_start_time = time.time()
for epoch in range(num_epochs):
    start_time = time.time()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    end_time = time.time()
    training_time = end_time - start_time

    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Training Time: {training_time:.2f} seconds')

total_end_time = time.time()
total_training_time = total_end_time - total_start_time
print(f'Total Training Time: {total_training_time:.2f} seconds')

# Testing the model
model.eval()
correct = 0
total = 0
all_predicted = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = correct / total
print(f'Final Test Accuracy: {accuracy * 100:.2f}%')

# Calculate and print F1 score, precision, and recall at the end
f1 = f1_score(all_labels, all_predicted, average='weighted')
precision = precision_score(all_labels, all_predicted, average='weighted')
recall = recall_score(all_labels, all_predicted, average='weighted')

print(f'Final F1 Score: {f1:.4f}')
print(f'Final Precision: {precision:.4f}')
print(f'Final Recall: {recall:.4f}')

# Calculate and print confusion matrix at the end
cm = confusion_matrix(all_labels, all_predicted)
print('Final Confusion Matrix:')
print(cm)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Epoch 1/300, Loss: 1.8251291940279324, Training Time: 49.22 seconds
Epoch 11/300, Loss: 0.6921585613428174, Training Time: 48.03 seconds
Epoch 21/300, Loss: 0.205784567043452, Training Time: 49.06 seconds
Epoch 31/300, Loss: 0.03428162000192058, Training Time: 48.17 seconds
Epoch 41/300, Loss: 0.013138540848658498, Training Time: 48.52 seconds
Epoch 51/300, Loss: 0.009550689951852535, Training Time: 39.53 seconds
Epoch 61/300, Loss: 0.005800260849224994, Training Time: 39.48 seconds
Epoch 71/300, Loss: 0.0040980548211706855, Training Time: 39.54 seconds
Epoch 81/300, Loss: 0.0038474751733894915, Training Time: 39.46 seconds
Epoch 91/300, Loss: 0.006307307020341263, Training Time: 39.42 seconds
Epoch 101/300, Loss: 0.0027406423845711876, Training Time: 39.39 seconds
Epoch 111/300, Loss: 0.008143035434064387, Training Time: 39.40 seconds
Epoch 121/300, Loss: 0.003394888263032

Dropout with p = 0.3

In [7]:
# Load CIFAR-10 dataset to calculate mean and std
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())

# Calculate mean and std
imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformation with calculated mean and std
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Load CIFAR-10 dataset with normalization
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, dropout_prob=0.3):
        super(ResidualBlock, self).__init__()

        # First convolutional layer
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_prob)

        # Second convolutional layer
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.dropout2 = nn.Dropout(dropout_prob)

        # Shortcut connection
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.Dropout(dropout_prob)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.dropout1(out)
        out = self.bn2(self.conv2(out))
        out = self.dropout2(out)
        out += self.shortcut(x)
        out = self.relu(out)
        return out

# Define ResNet-10 model
class ResNet10(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10, n_chans1=64, dropout_prob=0.3):
        super(ResNet10, self).__init__()
        self.in_channels = n_chans1

        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(n_chans1)
        self.relu = nn.ReLU()
        self.layer1 = self._make_layer(block, n_chans1, num_blocks[0], stride=1, dropout_prob=dropout_prob)
        self.layer2 = self._make_layer(block, n_chans1 * 2, num_blocks[1], stride=2, dropout_prob=dropout_prob)
        self.layer3 = self._make_layer(block, n_chans1 * 4, num_blocks[2], stride=2, dropout_prob=dropout_prob)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(n_chans1 * 4, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride, dropout_prob):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride, dropout_prob))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# Instantiate the ResNet-10 model with ResNet blocks
model = ResNet10(ResidualBlock, [4, 3, 3], dropout_prob=0.3).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Training loop
num_epochs = 300
total_start_time = time.time()
for epoch in range(num_epochs):
    start_time = time.time()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    end_time = time.time()
    training_time = end_time - start_time

    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Training Time: {training_time:.2f} seconds')

total_end_time = time.time()
total_training_time = total_end_time - total_start_time
print(f'Total Training Time: {total_training_time:.2f} seconds')

# Testing the model
model.eval()
correct = 0
total = 0
all_predicted = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = correct / total
print(f'Final Test Accuracy: {accuracy * 100:.2f}%')

# Calculate and print F1 score, precision, and recall at the end
f1 = f1_score(all_labels, all_predicted, average='weighted')
precision = precision_score(all_labels, all_predicted, average='weighted')
recall = recall_score(all_labels, all_predicted, average='weighted')

print(f'Final F1 Score: {f1:.4f}')
print(f'Final Precision: {precision:.4f}')
print(f'Final Recall: {recall:.4f}')

# Calculate and print confusion matrix at the end
cm = confusion_matrix(all_labels, all_predicted)
print('Final Confusion Matrix:')
print(cm)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Epoch 1/300, Loss: 2.1646225844197873, Training Time: 41.76 seconds
Epoch 11/300, Loss: 1.4325516841295736, Training Time: 41.62 seconds
Epoch 21/300, Loss: 1.1517963039753076, Training Time: 41.58 seconds
Epoch 31/300, Loss: 0.9842802292245734, Training Time: 41.65 seconds
Epoch 41/300, Loss: 0.8802655400217646, Training Time: 41.77 seconds
Epoch 51/300, Loss: 0.8018939947075856, Training Time: 41.39 seconds
Epoch 61/300, Loss: 0.7375333729150045, Training Time: 41.41 seconds
Epoch 71/300, Loss: 0.6801815891585996, Training Time: 41.43 seconds
Epoch 81/300, Loss: 0.6323559718287509, Training Time: 41.39 seconds
Epoch 91/300, Loss: 0.5870988661675807, Training Time: 41.36 seconds
Epoch 101/300, Loss: 0.5508497335645549, Training Time: 41.40 seconds
Epoch 111/300, Loss: 0.515632739952763, Training Time: 41.38 seconds
Epoch 121/300, Loss: 0.4933827482449734, Training Time: 41

Batch Normalization

In [ ]:
# Load CIFAR-10 dataset to calculate mean and std
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())

# Calculate mean and std
imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformation with calculated mean and std
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Load CIFAR-10 dataset with normalization
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()

        # First convolutional layer
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

        # Second convolutional layer
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Shortcut connection
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

# Define ResNet-10 model
class ResNet10(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10, n_chans1=64):
        super(ResNet10, self).__init__()
        self.in_channels = n_chans1

        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(n_chans1)
        self.relu = nn.ReLU()
        self.layer1 = self._make_layer(block, n_chans1, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, n_chans1 * 2, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, n_chans1 * 4, num_blocks[2], stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(n_chans1 * 4, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# Instantiate the ResNet-10 model with ResNet blocks and batch normalization
model = ResNet10(ResidualBlock, [4, 3, 3]).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Training loop
num_epochs = 300
total_start_time = time.time()
for epoch in range(num_epochs):
    start_time = time.time()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    end_time = time.time()
    training_time = end_time - start_time

    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Training Time: {training_time:.2f} seconds')

total_end_time = time.time()
total_training_time = total_end_time - total_start_time
print(f'Total Training Time: {total_training_time:.2f} seconds')

# Testing the model
model.eval()
correct = 0
total = 0
all_predicted = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = correct / total
print(f'Final Test Accuracy: {accuracy * 100:.2f}%')

# Calculate and print F1 score, precision, and recall at the end
f1 = f1_score(all_labels, all_predicted, average='weighted')
precision = precision_score(all_labels, all_predicted, average='weighted')
recall = recall_score(all_labels, all_predicted, average='weighted')

print(f'Final F1 Score: {f1:.4f}')
print(f'Final Precision: {precision:.4f}')
print(f'Final Recall: {recall:.4f}')

# Calculate and print confusion matrix at the end
cm = confusion_matrix(all_labels, all_predicted)
print('Final Confusion Matrix:')
print(cm)